# Libraries

In [105]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)

In [106]:
df = pd.read_csv('../data/fps-dataset-modified.csv', low_memory=False)
df.head()

,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuFrequency,CpuMultiplier,CpuMultiplierUnlocked,CpuProcessSize,CpuTDP,CpuTurboClock,GpuName,GpuArchitecture,GpuBandwidth,GpuBaseClock,GpuBoostClock,GpuBus nterface,GpuDieSize,GpuDirectX,GpuFP32Performance,GpuMemoryBus,GpuMemorySize,GpuMemoryType,GpuOpenCL,GpuOpenGL,GpuPixelRate,GpuProcessSize,GpuNumberOfROPs,GpuShaderModel,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,Dataset,FPS,GpuCpuRatio,CacheCoreRatio,EffectiveMemory,PerformanceClockRatio,PerformancePerWatt
0,Intel Core i7-920,4.0,8.0,133.0,256.0,1024.0,8.0,2666.0,20.0,0,45.0,130.0,2933.0,AMD Radeon RX 480,GCN 4.0,256000.0,1120.0,1266.0,PCIe 3.0 x16,0.000232,12,5834000.0,256.0,8000.0,GDDR5,2,4.6,40510.0,14.0,32.0,6.4,2304.0,144.0,182300.0,5700.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,70.0,0.431640,322.0,65536000.0,2.316746,1023.508772
1,Intel Core i7-920,4.0,8.0,133.0,256.0,1024.0,8.0,2666.0,20.0,0,45.0,130.0,2933.0,AMD Radeon RX 480,GCN 4.0,256000.0,1120.0,1266.0,PCIe 3.0 x16,0.000232,12,5834000.0,256.0,8000.0,GDDR5,2,4.6,40510.0,14.0,32.0,6.4,2304.0,144.0,182300.0,5700.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,230.0,0.431640,322.0,65536000.0,2.316746,1023.508772
2,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,250.0,0.456053,321.5,90188800.0,2.192729,1232.361111
3,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,300.0,0.456053,321.5,90188800.0,2.192729,1232.361111
4,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,AMD Radeon R7 260X,GCN 2.0,104000.0,1100.0,1100.0,PCIe 3.0 x16,0.000160,12,1971000.0,128.0,2000.0,GDDR5,2,4.6,17600.0,28.0,16.0,6.3,896.0,56.0,61600.0,2080.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,200.0,0.289474,321.5,13312000.0,3.454545,947.596154


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194700 entries, 0 to 194699
Data columns (total 46 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   CpuName                  194700 non-null  object 
 1   CpuNumberOfCores         194700 non-null  float64
 2   CpuNumberOfThreads       194700 non-null  float64
 3   CpuBaseClock             194700 non-null  float64
 4   CpuCacheL1               194700 non-null  float64
 5   CpuCacheL2               194700 non-null  float64
 6   CpuCacheL3               194700 non-null  float64
 7   CpuFrequency             194700 non-null  float64
 8   CpuMultiplier            194700 non-null  float64
 9   CpuMultiplierUnlocked    194700 non-null  int64  
 10  CpuProcessSize           194700 non-null  float64
 11  CpuTDP                   194700 non-null  float64
 12  CpuTurboClock            194700 non-null  float64
 13  GpuName                  194700 non-null  object 
 14  GpuA

In [108]:
# Verify if exists any duplicated row
df[df.duplicated()]

,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuFrequency,CpuMultiplier,CpuMultiplierUnlocked,CpuProcessSize,CpuTDP,CpuTurboClock,GpuName,GpuArchitecture,GpuBandwidth,GpuBaseClock,GpuBoostClock,GpuBus nterface,GpuDieSize,GpuDirectX,GpuFP32Performance,GpuMemoryBus,GpuMemorySize,GpuMemoryType,GpuOpenCL,GpuOpenGL,GpuPixelRate,GpuProcessSize,GpuNumberOfROPs,GpuShaderModel,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,Dataset,FPS,GpuCpuRatio,CacheCoreRatio,EffectiveMemory,PerformanceClockRatio,PerformancePerWatt


In [109]:
# Verify if exists any row with NaN values
df[df.isnull().any(axis=1)]

,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuFrequency,CpuMultiplier,CpuMultiplierUnlocked,CpuProcessSize,CpuTDP,CpuTurboClock,GpuName,GpuArchitecture,GpuBandwidth,GpuBaseClock,GpuBoostClock,GpuBus nterface,GpuDieSize,GpuDirectX,GpuFP32Performance,GpuMemoryBus,GpuMemorySize,GpuMemoryType,GpuOpenCL,GpuOpenGL,GpuPixelRate,GpuProcessSize,GpuNumberOfROPs,GpuShaderModel,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,Dataset,FPS,GpuCpuRatio,CacheCoreRatio,EffectiveMemory,PerformanceClockRatio,PerformancePerWatt


PCA is used in this case to reduce dimensionality while retaining the most important variance in the dataset. Given the high number of features related to CPU and GPU specifications, PCA helps to simplify the data without losing significant information, making it easier to visualize and analyze. By transforming the original correlated features into a smaller set of uncorrelated components, PCA also helps mitigate the risk of overfitting in machine learning models and improves computational efficiency.

In [110]:
# Select only numeric features
numeric_features = df.select_dtypes(include=['float64', 'int64']).columns

# Scale the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[numeric_features])

# Create a PCA object with 2 components
pca = PCA(n_components=2) 
principal_components = pca.fit_transform(scaled_data)

# Create a DataFrame with the principal components
pca_df = pd.DataFrame(data=principal_components, columns=[f'PC{i+1}' for i in range(principal_components.shape[1])])
pca_df

,PC1,PC2
0,-0.976108,-0.285104
1,-0.785587,-0.331431
2,3.004209,-2.526221
3,3.063747,-2.540699
4,-3.770181,-1.075099
...,...,...
194695,0.629717,1.165130
194696,0.344657,1.386445
194697,0.280569,1.229927
194698,0.139496,0.724430


In [111]:
# Verify quantity of columns and rows
df.shape

(194700, 46)

In [112]:
# Verify that PCA has 2 columns
pca_df.shape

(194700, 2)